# Image Captioning - Model Training

In this notebook, we will train the image captioning model using the pre-extracted features from the CNN encoder. We will:

1. Set up the data loaders for training and validation
2. Build the caption model (combining the encoder and decoder)
3. Define the training pipeline with teacher forcing
4. Train the model with appropriate hyperparameters
5. Monitor the training progress and validation performance
6. Save the trained model for later evaluation

In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import time
import h5py

# Add project root to path
sys.path.append('..')

# Import project modules
from models.encoder import EncoderCNN
from models.decoder import DecoderRNN
from models.caption_model import CaptionModel
from utils.vocabulary import Vocabulary
from utils.dataset import get_data_loaders
from utils.trainer import CaptionTrainer

## 1. Configuration Settings

In [8]:
# Define configuration settings
config = {
    # Data settings
    'data_dir': '../data/flickr8k',
    'batch_size': 32,
    'num_workers': 4,
    
    # Model settings
    'encoder_model': 'resnet18',  # Options: 'resnet18', 'resnet50', 'mobilenet_v2'
    'embed_size': 256,
    'hidden_size': 512,
    'num_layers': 1,
    'dropout': 0.5,
    'decoder_type': 'lstm',  # Options: 'lstm', 'gru'
    
    # Training settings
    'learning_rate': 3e-4,
    'num_epochs': 2,  # Increase for better results
    'early_stopping_patience': 5,
    'save_dir': '../models',
    
    # Device settings
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Create save directory if it doesn't exist
os.makedirs(config['save_dir'], exist_ok=True)

# Print configuration
for section, items in {
    'Data': ['data_dir', 'batch_size', 'num_workers'],
    'Model': ['encoder_model', 'embed_size', 'hidden_size', 'num_layers', 'dropout', 'decoder_type'],
    'Training': ['learning_rate', 'num_epochs', 'early_stopping_patience', 'save_dir'],
    'Device': ['device']
}.items():
    print(f"\n{section} settings:")
    for item in items:
        print(f"  {item}: {config[item]}")


Data settings:
  data_dir: ../data/flickr8k
  batch_size: 32
  num_workers: 4

Model settings:
  encoder_model: resnet18
  embed_size: 256
  hidden_size: 512
  num_layers: 1
  dropout: 0.5
  decoder_type: lstm

Training settings:
  learning_rate: 0.0003
  num_epochs: 2
  early_stopping_patience: 5
  save_dir: ../models

Device settings:
  device: cpu


## 2. Load Data

In [9]:
# Get data loaders
train_loader, val_loader, test_loader, vocab = get_data_loaders(
    data_dir=config['data_dir'],
    batch_size=config['batch_size'],
    num_workers=config['num_workers']
)

# Update vocabulary size in config
config['vocab_size'] = len(vocab)
print(f"Vocabulary size: {config['vocab_size']}")

# Display data loader information
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Vocabulary size: 2986
Training batches: 937
Validation batches: 157
Test batches: 5000


## 3. Create Model

In [10]:
from torchinfo import summary

In [11]:
# Create caption model
model = CaptionModel(
    embed_size=config['embed_size'],
    hidden_size=config['hidden_size'],
    vocab_size=config['vocab_size'],
    num_layers=config['num_layers'],
    encoder_model=config['encoder_model'],
    decoder_type=config['decoder_type'],
    dropout=config['dropout'],
    train_encoder=False  # Don't train the encoder (use pre-trained weights)
)

# Print model architecture
# print(model)
summary(model, input_size=[(1, 3, 128, 128), (1, 20)], dtypes=[torch.float, torch.long])

d:\Aanaconda\envs\image_captioning\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Layer (type:depth-idx)                             Output Shape              Param #
CaptionModel                                       [1, 19, 2986]             --
├─EncoderCNN: 1-1                                  [1, 256]                  --
│    └─ResNet: 2-1                                 [1, 512]                  --
│    │    └─Conv2d: 3-1                            [1, 64, 64, 64]           (9,408)
│    │    └─BatchNorm2d: 3-2                       [1, 64, 64, 64]           (128)
│    │    └─ReLU: 3-3                              [1, 64, 64, 64]           --
│    │    └─MaxPool2d: 3-4                         [1, 64, 32, 32]           --
│    │    └─Sequential: 3-5                        [1, 64, 32, 32]           (147,968)
│    │    └─Sequential: 3-6                        [1, 128, 16, 16]          (525,568)
│    │    └─Sequential: 3-7                        [1, 256, 8, 8]            (2,099,712)
│    │    └─Sequential: 3-8                        [1, 512, 4, 4]            (8,393,

In [12]:
# Calculate model size
def count_parameters(model: nn.Module):
    """Count the number of trainable parameters in a model."""
    num_of_trainable_parameters = sum([param.numel() for param in model.parameters() if param.requires_grad])
    return num_of_trainable_parameters

# Calculate and print model size
num_params = count_parameters(model)
print(f"Number of trainable parameters: {num_params}")

# Calculate encoder and decoder sizes separately
encoder_params = count_parameters(model.encoder)
decoder_params = count_parameters(model.decoder)
print(f"Encoder parameters: {encoder_params:,} ({encoder_params / num_params * 100:.1f}%)")
print(f"Decoder parameters: {decoder_params:,} ({decoder_params / num_params * 100:.1f}%)")

Number of trainable parameters: 4136618
Encoder parameters: 131,840 (3.2%)
Decoder parameters: 4,004,778 (96.8%)


## 4. Train the Model

In [13]:
# Create trainer
trainer = CaptionTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    vocab=vocab,
    device=config['device'],
    learning_rate=config['learning_rate'],
    model_save_dir=config['save_dir']
)

In [14]:
# Train the model
history = trainer.train(
    epochs=config['num_epochs'],
    early_stopping_patience=config['early_stopping_patience'],
    evaluate_every=1,
    generate_every=1  # Generate captions and calculate BLEU every 5 epochs
)

Starting training for 2 epochs...
Training on device: cpu


d:\Aanaconda\envs\image_captioning\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

## 5. Analyze Training Results

In [ ]:
# Plot training history
fig, axs = trainer.plot_history()
plt.show()

In [ ]:
# Find best epoch
best_epoch = np.argmin(trainer.history['val_loss'])
print(f"Best epoch: {best_epoch + 1}")
print(f"Best validation loss: {trainer.history['val_loss'][best_epoch]:.4f}")

# If BLEU scores were calculated
bleu_epochs = [i for i, bleu in enumerate(trainer.history['val_bleu']) if bleu > 0]
if bleu_epochs:
    best_bleu_epoch = bleu_epochs[np.argmax([trainer.history['val_bleu'][i] for i in bleu_epochs])]
    print(f"Best BLEU epoch: {best_bleu_epoch + 1}")
    print(f"Best BLEU score: {trainer.history['val_bleu'][best_bleu_epoch]:.4f}")

## 6. Generate Sample Captions

In [ ]:
# Load the best model
best_model_path = os.path.join(config['save_dir'], 'best_model_loss.pth')
trainer.load_checkpoint(best_model_path)
model = trainer.model.to(config['device'])

In [ ]:
# Generate captions for some validation images
def generate_caption(image, model, vocab, device):
    """Generate a caption for an image."""
    
    caption_text = ...
    # TODO: Implement the caption generation function
    # 1. Set the model to evaluation mode
    # 2. Use torch.no_grad() to disable gradient calculation during inference
    # 3. Move the image to the device and add a batch dimension if needed
    # 4. Generate a caption using the model's generate_caption method
    # 5. Decode the caption indices to text using the vocabulary
    # 6. Return the caption text
    
    return caption_text

# Get some validation examples
num_examples = 5
val_examples = []

for images, captions, image_ids in val_loader:
    if len(val_examples) >= num_examples:
        break
    
    # Generate captions
    for i in range(len(images)):
        if len(val_examples) >= num_examples:
            break
        
        image = images[i]
        true_caption = vocab.decode(captions[i], join=True, remove_special=True)
        generated_caption = generate_caption(image, model, vocab, config['device'])
        
        val_examples.append({
            'image': image,
            'image_id': image_ids[i],
            'true_caption': true_caption,
            'generated_caption': generated_caption
        })

# Display examples
plt.figure(figsize=(15, 5 * num_examples))

for i, example in enumerate(val_examples):
    plt.subplot(num_examples, 1, i + 1)
    
    # Convert tensor to image
    img = example['image'].permute(1, 2, 0).numpy()
    img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]  # Denormalize
    img = np.clip(img, 0, 1)
    
    plt.imshow(img)
    plt.title(f"Image: {example['image_id']}")
    plt.axis('off')
    plt.figtext(0.5, 0.01 + i * (1/num_examples), f"True caption: {example['true_caption']}", 
                ha='center', fontsize=12, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    plt.figtext(0.5, 0.05 + i * (1/num_examples), f"Generated caption: {example['generated_caption']}", 
                ha='center', fontsize=12, bbox={"facecolor":"lightgreen", "alpha":0.5, "pad":5})

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## 7. Compare Greedy and Beam Search Decoding

In [ ]:
# Compare greedy decoding vs. beam search
def compare_decoding_methods(image, model, vocab, device, beam_sizes=[1, 3, 5]):
    """Compare different beam search sizes for caption generation."""
    results = ...
    # TODO: Implement a function to compare different beam search settings
    # 1. Set the model to evaluation mode
    # 2. Initialize a dictionary to store results
    # 3. For each beam size:
    #    a. Generate a caption using that beam size
    #    b. Decode the caption indices to text
    #    c. Store the result in the dictionary
    # 4. Return the dictionary of results
    
    return results

# Select a random example from val_examples
import random
example = random.choice(val_examples)

# Compare decoding methods
beam_results = compare_decoding_methods(
    example['image'], 
    model, 
    vocab, 
    config['device'], 
    beam_sizes=[1, 3, 5]
)

# Display the image with different captions
plt.figure(figsize=(10, 10))

# Display image
img = example['image'].permute(1, 2, 0).numpy()
img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]  # Denormalize
img = np.clip(img, 0, 1)
plt.imshow(img)
plt.axis('off')
plt.title(f"Image: {example['image_id']}")

# Add captions
captions = [
    f"True caption: {example['true_caption']}",
    f"Greedy search: {beam_results['beam_1']}",
    f"Beam search (k=3): {beam_results['beam_3']}",
    f"Beam search (k=5): {beam_results['beam_5']}"
]

# Display captions below the image
plt.figtext(0.5, 0.01, '\n'.join(captions), ha='center', fontsize=12, 
            bbox={"facecolor":"white", "alpha":0.8, "pad":5})

plt.tight_layout(rect=[0, 0.2, 1, 0.95])
plt.show()

## 8. Save Final Model and Configuration

In [ ]:
# Save configuration
import json
config_path = os.path.join(config['save_dir'], 'config.json')

# Convert non-serializable values to strings
serializable_config = {k: str(v) if not isinstance(v, (int, float, str, bool)) else v 
                      for k, v in config.items()}

with open(config_path, 'w') as f:
    json.dump(serializable_config, f, indent=4)

print(f"Configuration saved to {config_path}")

In [ ]:
# Save vocabulary separately for easy access
import pickle
vocab_path = os.path.join(config['save_dir'], 'vocabulary.pkl')
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)

print(f"Vocabulary saved to {vocab_path}")

## Summary

In this notebook, we have successfully:

1. Set up the data loaders for training and validation
2. Built the caption model by combining the encoder and decoder
3. Trained the model with teacher forcing
4. Monitored the training progress and validation performance
5. Generated captions for sample images
6. Compared different decoding strategies (greedy vs. beam search)
7. Saved the model, configuration, and vocabulary for later use

In the next notebook, we will perform a comprehensive evaluation of the model on the test set.